In [2]:
import sys
import glob
sys.path.append("../../")
sys.path.append("../../experiments/params_save/")

from python.src.train import experiment_tracking_train
from python.src.sat_representations import VCG, LCG, SATRepresentation
from pathlib import Path
NUM_EPOCHS = 5  # 10
f = 0.0000001
alpha = 1
beta = 1
gamma = 0
batch_size = 3
# path = "/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/GIT_SAT_ML/data/blocksworld"
path = "/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/generateSAT/random_3SAT_subset"
# path = "../../Data/blocksworld"
# path = "/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/GIT_SAT_ML/data/LLL_sample_one"
# path = "/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/generateSAT/samples_LLL_n80/"
N_STEPS_MOSER = 1000
N_RUNS_MOSER = 5
SEED = 0
graph_representation = "VCG"
network_type = "interaction"
return_candidates = True
# network_definition = get_network_definition(network_type = network_type, graph_representation = graph_representation) #network_definition_interaction_new

MODEL_REGISTRY = Path("../../../mlrun_save")
EXPERIMENT_NAME = "blocksworld_LCG_interaction"

model_path = "../../../params_save/" + EXPERIMENT_NAME
img_path = False # model_path + "_plot"

match graph_representation:
    case "LCG":
            rep = LCG
    case "VCG":
            rep = VCG
print(rep)
print(glob.glob(glob.escape(path) + "/*.cnf"))


experiment_tracking_train(
    MODEL_REGISTRY,
    EXPERIMENT_NAME,
    batch_size,
    f,
    alpha,
    beta,
    gamma,
    NUM_EPOCHS,
    N_STEPS_MOSER,
    N_RUNS_MOSER,
    path,
    "VCG",
    network_type = network_type,
    return_candidates=True,
)

<class 'python.src.sat_representations.VCG'>
['/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/generateSAT/random_3SAT_subset/random_KCNF3_100_119_9283351.cnf', '/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/generateSAT/random_3SAT_subset/random_KCNF3_100_119_6639307.cnf', '/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/generateSAT/random_3SAT_subset/random_KCNF3_100_119_4391198.cnf', '/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/generateSAT/random_3SAT_subset/random_KCNF3_100_100_3602064.cnf', '/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/generateSAT/random_3SAT_subset/random_KCNF3_100_100_2658295.cnf', '/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/generateSAT/random_3SAT_subset/random_KCNF3_100_100_2924554.cnf']


FileNotFoundError: [Errno 2] No such file or directory: 'experiments/params_save'

In [6]:
import numpy as np
x = np.array([1,1,1,0,0])
np.array(np.logical_not(x),dtype = int)

array([0, 0, 0, 1, 1])